In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [975 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,59

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)
df.show()


+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa0052

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT round(avg(price),2) FROM home_sales WHERE bedrooms=4").show()


+--------------------+
|round(avg(price), 2)|
+--------------------+
|           299661.01|
+--------------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("SELECT round(avg(price),2) FROM home_sales WHERE bedrooms=3 AND bathrooms=3").show()


+--------------------+
|round(avg(price), 2)|
+--------------------+
|           292003.61|
+--------------------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql('SELECT round(avg(price),2) FROM home_sales WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living >=2000').show()


+--------------------+
|round(avg(price), 2)|
+--------------------+
|           292538.42|
+--------------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT view FROM home_sales WHERE price >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+
|view|
+----+
|  76|
|  23|
|   0|
|   4|
|   6|
|   7|
|  26|
|  34|
|  23|
|  48|
|  16|
|  10|
|  24|
|  91|
|  49|
|  37|
|  23|
|  25|
|  23|
|   1|
+----+
only showing top 20 rows

--- 0.36376094818115234 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price of greater than $350,000. 
# Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT view FROM home_sales WHERE price >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+
|view|
+----+
|  76|
|  23|
|   0|
|   4|
|   6|
|   7|
|  26|
|  34|
|  23|
|  48|
|  16|
|  10|
|  24|
|  91|
|  49|
|  37|
|  23|
|  25|
|  23|
|   1|
+----+
only showing top 20 rows

--- 0.2540709972381592 seconds ---


In [12]:
# 10. Partition the home sales dataset by the date_built field.
df.write.partitionBy('date_built').parquet('home_sales', mode='overwrite')

In [13]:
# 11. Read the formatted parquet data.
p_df = spark.read.parquet('home_sales')

In [14]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('home_sales')

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than $350,000 with the parquet DataFrame. 
# Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

spark.sql("SELECT view FROM home_sales WHERE price >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+
|view|
+----+
|  10|
|  24|
|   1|
|   4|
|  39|
|  90|
|  50|
|   6|
|   0|
|  26|
|  17|
|  10|
|  58|
|  82|
|   2|
|  44|
|  45|
|  88|
|  49|
|  56|
+----+
only showing top 20 rows

--- 0.5286953449249268 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [19]:
# 15. Check if the home_sales is no longer cached

if spark.catalog.isCached('home_sales'):
  print('home_sales is cached')
else:
  print('not cached')

not cached


In [18]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False